In [2]:
import pandas as pd
import tensorflow as tf
import keras
from keras import layers
import os
import numpy as np

2024-08-22 19:25:49.524295: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-22 19:25:49.620245: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-22 19:25:49.648001: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-22 19:25:49.801980: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-22 19:25:51.505546: W tensorflow/compiler/tf2

In [3]:
DATA_PATH = "../backend/src/data/"
df = pd.read_csv(DATA_PATH+'job_demand_30.csv', index_col=0)
df.head(3)

,Artificial Intelligence/Machine Learning (AI/ML) Engineer,Business Analyst,Cloud Engineer,Cybersecurity Analyst,Data Engineer,Data Scientist,Database Administrator (DBA),DevOps Engineer,IT Consultant,IT Manager,IT Project Manager,IT Support Specialist,Information Security Manager,Network Administrator,Quality Assurance (QA) Tester/Engineer,Software Developer/Engineer,Systems Analyst,Systems Architect,Technical Support Engineer,Web Developer
2021-12-12,0.266667,6.533333,1.200000,1.1,5.133333,2.900000,1.3,3.033333,9.966667,12.866667,5.200000,4.566667,2.500000,2.733333,3.766667,20.833333,2.7,2.966667,8.800000,17.866667
2021-12-13,0.266667,6.966667,1.266667,1.1,5.200000,3.066667,1.3,3.200000,10.466667,13.100000,5.366667,4.766667,2.566667,2.766667,3.900000,21.666667,2.8,3.000000,9.200000,18.133333
2021-12-14,0.266667,7.166667,1.366667,1.1,5.233333,3.233333,1.3,3.466667,10.600000,13.433333,5.466667,4.900000,2.666667,2.833333,4.033333,22.433333,2.9,3.066667,9.566667,18.133333


We'll experiment using the Software Engineer column first and then we try the others

# Model Experiments

1. Vanilla-LSTM
2. N-BEATS
3. Ensemble Model 

In [69]:
def mean_absolute_scaled_error(y_true, y_pred):
    """
    Implement MASE (assuming no seasonality of data).
    """
    mae = tf.reduce_mean(tf.abs(y_true - y_pred))

    # Find MAE of naive forecast (no seasonality)
    mae_naive_no_season = tf.reduce_mean(tf.abs(y_true[1:] - y_true[:-1])) # our seasonality is 1 day (hence the shifting of 1 day)

    return mae / mae_naive_no_season

In [70]:
def evaluate_preds(y_true, y_pred):
    # Make sure float32 (for metric calculations)
    y_true = tf.cast(y_true, dtype=tf.float32)
    y_pred = tf.cast(y_pred, dtype=tf.float32)

    # Calculate various metrics
    mae = keras.metrics.mean_absolute_error(y_true, y_pred)
    mse = keras.metrics.mean_squared_error(y_true, y_pred) # puts and emphasis on outliers (all errors get squared)
    rmse = tf.sqrt(mse)
    mape = keras.metrics.mean_absolute_percentage_error(y_true, y_pred)
    mase = mean_absolute_scaled_error(y_true, y_pred)
    
    return {"mae": tf.reduce_mean(mae.numpy()),
            "mse": tf.reduce_mean(mse.numpy()),
            "rmse": tf.reduce_mean(rmse.numpy()),
            "mape": tf.reduce_mean(mape.numpy()),
            "mase": tf.reduce_mean(mase.numpy())}

In [71]:
se_demand = df['Software Developer/Engineer'].to_numpy()

In [72]:
def get_labelled_windows(x, horizon=1):
  return x[:, :-horizon], x[:, -horizon:]

def make_windows(x, window_size=7, horizon=1):
  """
  Turns a 1D array into a 2D array of sequential windows of window_size.
  """
  window_step = np.expand_dims(np.arange(window_size+horizon), axis=0)

  window_indexes = window_step + np.expand_dims(np.arange(len(x)-(window_size+horizon-1)), axis=0).T

  windowed_array = x[window_indexes]

  windows, labels = get_labelled_windows(windowed_array, horizon=horizon)

  return windows, labels

In [73]:
WINDOW_SIZE = 60
HORIZON = 30
full_windows, full_labels = make_windows(se_demand, window_size=60, horizon=30)

In [74]:
def make_train_test_splits(windows, labels, test_split=0.2):
    """
    Splits matching pairs of windows and labels into train and test splits.
    """
    split_size = int(len(windows) * (1-test_split)) # this will default to 80% train/20% test
    train_windows = windows[:split_size]
    train_labels = labels[:split_size]
    test_windows = windows[split_size:]
    test_labels = labels[split_size:]
    return train_windows, test_windows, train_labels, test_labels

In [75]:
train_windows, test_windows, train_labels, test_labels = make_train_test_splits(full_windows, full_labels)
len(train_windows), len(test_windows), len(train_labels), len(test_labels)

(138, 35, 138, 35)

In [76]:
def create_model_checkpoint(model_name, save_path="model_experiments"):
    return keras.callbacks.ModelCheckpoint(filepath=os.path.join(save_path, model_name), # create filepath to save model
                                                verbose=0, # only output a limited amount of text
                                                save_best_only=True)

## Model 0: LSTM

In [77]:
tf.random.set_seed(42)

def transform(x):
    import tensorflow as tf
    return tf.expand_dims(x, axis=1)

inputs = layers.Input(shape=[WINDOW_SIZE])
x = layers.Lambda(transform)(inputs)
x = layers.LSTM(128, activation='relu')(x)
output = layers.Dense(HORIZON)(x)
model_0 = keras.Model(inputs=inputs, outputs=output, name='model_0_lstm')

model_0.compile(loss='mae',
                optimizer=keras.optimizers.Adam())

model_0.fit(train_windows,
            train_labels,
            epochs=100,
            verbose=0,
            batch_size=32,
            validation_data=(test_windows, test_labels),
            callbacks=[create_model_checkpoint(model_name=model_0.name+'.keras')])

In [78]:
model_0 = keras.models.load_model("model_experiments/model_0_lstm.keras",
                                  custom_objects={'transform': transform},
                                  safe_mode=False)
model_0.evaluate(test_windows, test_labels)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 11.2903  


11.429537773132324

In [79]:
# Make predictions with our LSTM model
model_0_preds = model_0(test_windows)
model_0_preds[:10]

<tf.Tensor: shape=(10, 30), dtype=float32, numpy=
array([[ 98.58131 ,  97.89175 ,  98.70699 ,  99.11427 , 101.08131 ,
         98.70045 ,  99.369354, 102.08051 , 101.547775, 101.116066,
        102.921684, 106.82257 , 102.593704, 105.36856 , 104.79472 ,
        103.59391 , 100.82244 , 102.893936, 104.83956 ,  99.28092 ,
         97.217026, 100.783745, 100.89875 ,  99.09374 ,  98.41969 ,
        100.30977 ,  99.7244  ,  97.795494,  99.57678 ,  98.54738 ],
       [ 98.17683 ,  97.03095 ,  98.040306,  98.66959 , 100.158066,
         97.79937 ,  99.090576, 102.199005, 100.79794 , 100.799904,
        102.8551  , 106.11826 , 101.67517 , 105.361046, 104.10607 ,
        102.82795 , 100.420425, 102.381836, 104.22042 ,  99.25396 ,
         96.95264 , 100.56793 , 100.66818 ,  98.22179 ,  98.658066,
         99.91066 ,  99.213646,  97.4239  ,  98.60182 ,  98.28718 ],
       [ 97.98418 ,  96.21365 ,  97.327385,  98.043015,  98.97754 ,
         96.81642 ,  98.9241  , 102.24777 , 100.12222 , 100.2839

In [80]:
model_0_results = evaluate_preds(y_true=tf.squeeze(test_labels),
                                 y_pred=model_0_preds)
model_0_results

{'mae': <tf.Tensor: shape=(), dtype=float32, numpy=11.429537>,
 'mse': <tf.Tensor: shape=(), dtype=float32, numpy=159.11499>,
 'rmse': <tf.Tensor: shape=(), dtype=float32, numpy=12.430287>,
 'mape': <tf.Tensor: shape=(), dtype=float32, numpy=14.108284>,
 'mase': <tf.Tensor: shape=(), dtype=float32, numpy=12.563094>}

# Model 1: N-BEATS Algorithm

In [81]:
class NBeatsBlock(keras.layers.Layer):
    def __init__(self,
                input_size: int,
                theta_size: int,
                horizon: int,
                n_neurons: int,
                n_layers: int,
                **kwargs):
        super().__init__(**kwargs)
        self.input_size = input_size
        self.theta_size = theta_size
        self.horizon = horizon
        self.n_neurons = n_neurons
        self.n_layers = n_layers

        # Block contains stack of 4 fully connected layers each has ReLU activation
        self.hidden = [tf.keras.layers.Dense(n_neurons, activation="relu") for _ in range(n_layers)]
        # Output of block is a theta layer with linear activation
        self.theta_layer = tf.keras.layers.Dense(theta_size, activation="linear", name="theta")

    def call(self, inputs): # the call method is what runs when the layer is called 
        x = inputs 
        for layer in self.hidden: # pass inputs through each hidden layer 
            x = layer(x)
        theta = self.theta_layer(x) 
        # Output the backcast and forecast from theta
        backcast, forecast = theta[:, :self.input_size], theta[:, -self.horizon:]
        return backcast, forecast

In [82]:
# 1. Turn tensor slices to dataset
train_features_dataset = tf.data.Dataset.from_tensor_slices(train_windows)
train_labels_dataset = tf.data.Dataset.from_tensor_slices(train_labels)

test_features_dataset = tf.data.Dataset.from_tensor_slices(test_windows)
test_labels_dataset = tf.data.Dataset.from_tensor_slices(test_labels)

# 2. Combine features and labels
train_dataset = tf.data.Dataset.zip((train_features_dataset, train_labels_dataset))
test_dataset = tf.data.Dataset.zip((test_features_dataset, test_labels_dataset))

# 3. Batch and prefetch for optimal performance
BATCH_SIZE = 32
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

train_dataset, test_dataset

(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 60), dtype=tf.float64, name=None), TensorSpec(shape=(None, 30), dtype=tf.float64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 60), dtype=tf.float64, name=None), TensorSpec(shape=(None, 30), dtype=tf.float64, name=None))>)

In [83]:
N_EPOCHS = 5000 
N_NEURONS = 512 
N_LAYERS = 4
N_STACKS = 30

INPUT_SIZE = WINDOW_SIZE * HORIZON 
THETA_SIZE = INPUT_SIZE + HORIZON

INPUT_SIZE, THETA_SIZE

(1800, 1830)

In [85]:
tf.random.set_seed(42)

# 1. Setup N-BEATS Block layer
nbeats_block_layer = NBeatsBlock(input_size=60,
                                 theta_size=THETA_SIZE,
                                 horizon=HORIZON,
                                 n_neurons=N_NEURONS,
                                 n_layers=N_LAYERS,
                                 name="InitialBlock")

# 2. Create input to stacks
stack_input = layers.Input(shape=[60], name="stack_input")

# 3. Create initial backcast and forecast input (backwards predictions are referred to as residuals in the paper)
backcast, forecast = nbeats_block_layer(stack_input)
residuals = layers.subtract([stack_input, backcast], name=f"subtract_00") 

# 4. Create stacks of blocks
for i, _ in enumerate(range(N_STACKS-1)): # first stack is already creted in (3)

    # 5. Use the NBeatsBlock to calculate the backcast as well as block forecast
    backcast, block_forecast = NBeatsBlock(
        input_size=60,
        theta_size=THETA_SIZE,
        horizon=HORIZON,
        n_neurons=N_NEURONS,
        n_layers=N_LAYERS,
        name=f"NBeatsBlock_{i}"
    )(residuals) # pass it in residuals (the backcast)

    # 6. Create the double residual stacking
    residuals = layers.subtract([residuals, backcast], name=f"subtract_{i}") 
    forecast = layers.add([forecast, block_forecast], name=f"add_{i}")

# 7. Put the stack model together
model_1 = keras.Model(inputs=stack_input, 
                         outputs=forecast, 
                         name="model_1_N-BEATS")

model_1.compile(loss="mae",
                optimizer=keras.optimizers.Adam(0.001),
                metrics=["mae", "mse"])

# 9. Fit the model with EarlyStopping and ReduceLROnPlateau callbacks
model_1.fit(train_dataset,
            epochs=N_EPOCHS,
            validation_data=test_dataset,
            verbose=1, # prevent large amounts of training outputs
            # callbacks=[create_model_checkpoint(model_name=stack_model.name)] # saving model every epoch consumes far too much time
            callbacks=[keras.callbacks.EarlyStopping(monitor="val_loss", patience=200, restore_best_weights=True),
                      keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=100, verbose=1)])
     

Epoch 1/5000
5/5 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - loss: 140.3158 - mae: 140.3158 - mse: 38002.9531 - val_loss: 14.8837 - val_mae: 14.8837 - val_mse: 367.2444 - learning_rate: 0.0010
Epoch 2/5000
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 595ms/step - loss: 86.5311 - mae: 86.5311 - mse: 19570.8281 - val_loss: 23.2300 - val_mae: 23.2300 - val_mse: 756.3934 - learning_rate: 0.0010
Epoch 3/5000
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 592ms/step - loss: 29.2403 - mae: 29.2403 - mse: 1187.4751 - val_loss: 10.9405 - val_mae: 10.9405 - val_mse: 178.1383 - learning_rate: 0.0010
Epoch 4/5000
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 585ms/step - loss: 19.2867 - mae: 19.2867 - mse: 484.6366 - val_loss: 9.6520 - val_mae: 9.6520 - val_mse: 172.3224 - learning_rate: 0.0010
Epoch 5/5000
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 586ms/step - loss: 16.9813 - mae: 16.9813 - mse: 388.1904 - val_loss: 15.4309 - val_mae: 15.4309 - val_mse: 298.3350 - learning_rate: 0.0010
Epoch 6/5000
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 575ms/step - loss: 11.4887 - mae: 11.4887 - ms

In [86]:
model_1.evaluate(test_dataset)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 5.7706 - mae: 5.7706 - mse: 67.4544


[6.047199726104736, 6.047199726104736, 73.7038803100586]

In [87]:
# Make predictions with N-BEATS model
model_1_preds = model_1.predict(test_dataset)

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step


In [88]:

# Evaluate N-BEATS model predictions
model_1_results = evaluate_preds(y_true=tf.squeeze(test_labels),
                                 y_pred=model_1_preds)
model_1_results

{'mae': <tf.Tensor: shape=(), dtype=float32, numpy=6.0471997>,
 'mse': <tf.Tensor: shape=(), dtype=float32, numpy=73.70388>,
 'rmse': <tf.Tensor: shape=(), dtype=float32, numpy=7.2709985>,
 'mape': <tf.Tensor: shape=(), dtype=float32, numpy=7.671045>,
 'mase': <tf.Tensor: shape=(), dtype=float32, numpy=6.6469483>}

# Model 2: Ensemble Model

In [89]:
def get_ensemble_models(horizon=HORIZON, 
                        train_data=train_dataset,
                        test_data=test_dataset,
                        num_iter=10, 
                        num_epochs=100, 
                        loss_fns=["mae", "mse", "mape"]):
    """
    Returns a list of num_iter models each trained on MAE, MSE and MAPE loss.

    For example, if num_iter=10, a list of 30 trained models will be returned:
    10 * len(["mae", "mse", "mape"]).
    """
    # Make empty list for trained ensemble models
    ensemble_models = []

    # Create num_iter number of models per loss function
    for i in range(num_iter):
        # Build and fit a new model with a different loss function
        for loss_function in loss_fns:
            print(f"Optimizing model by reducing: {loss_function} for {num_epochs} epochs, model number: {i}")

            # Construct a simple model (similar to model_1)
            model = tf.keras.Sequential([
                # Initialize layers with normal (Gaussian) distribution so we can use the models for prediction
                # interval estimation later: https://www.tensorflow.org/api_docs/python/tf/keras/initializers/HeNormal
                layers.Dense(128, kernel_initializer="he_normal", activation="relu"), 
                layers.Dense(128, kernel_initializer="he_normal", activation="relu"),
                layers.Dense(HORIZON)                                 
            ])

            # Compile simple model with current loss function
            model.compile(loss=loss_function,
                            optimizer=tf.keras.optimizers.Adam(),
                            metrics=["mae", "mse"])
            
            # Fit model
            model.fit(train_data,
                        epochs=num_epochs,
                        verbose=0,
                        validation_data=test_data,
                        # Add callbacks to prevent training from going/stalling for too long
                        callbacks=[keras.callbacks.EarlyStopping(monitor="val_loss",
                                                                    patience=200,
                                                                    restore_best_weights=True),
                                keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                                        patience=100,
                                                                        verbose=1)])
            
            # Append fitted model to list of ensemble models
            ensemble_models.append(model)

    return ensemble_models # return list of trained models

In [90]:
ensemble_models = get_ensemble_models(num_iter=5,
                                      num_epochs=1000)

Optimizing model by reducing: mae for 1000 epochs, model number: 0

Epoch 226: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 326: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
Optimizing model by reducing: mse for 1000 epochs, model number: 0

Epoch 176: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 276: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
Optimizing model by reducing: mape for 1000 epochs, model number: 0

Epoch 258: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 358: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
Optimizing model by reducing: mae for 1000 epochs, model number: 1

Epoch 207: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 307: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
Optimizing model by reducing: mse for 1000 epochs, model number: 1

Epoch 213: ReduceLROnPl

In [91]:
def make_ensemble_preds(ensemble_models, data):
    ensemble_preds = []
    for model in ensemble_models:
        preds = model.predict(data) # make predictions with current ensemble model
        ensemble_preds.append(preds)
    return tf.constant(tf.squeeze(ensemble_preds))

In [92]:
# Create a list of ensemble predictions
ensemble_preds = make_ensemble_preds(ensemble_models=ensemble_models,
                                     data=test_dataset)
ensemble_preds

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


<tf.Tensor: shape=(15, 35, 30), dtype=float32, numpy=
array([[[ 91.90076 ,  91.455284,  90.06599 , ...,  89.6185  ,
          83.25336 ,  82.04163 ],
        [ 94.000114,  92.84287 ,  87.92823 , ...,  87.28374 ,
          83.414246,  81.08657 ],
        [ 93.92259 ,  91.65942 ,  85.81731 , ...,  86.61658 ,
          82.65299 ,  80.29374 ],
        ...,
        [ 87.50764 ,  86.01962 ,  89.39275 , ..., 105.26368 ,
          89.239914,  94.85974 ],
        [ 87.32938 ,  85.21523 ,  89.686   , ..., 107.876205,
          89.74566 ,  96.59209 ],
        [ 87.16394 ,  85.161224,  90.284386, ..., 108.92953 ,
          91.231186,  98.93103 ]],

       [[ 95.24617 ,  92.195946,  90.41885 , ...,  80.643555,
          84.19835 ,  81.746605],
        [ 94.94705 ,  89.035736,  89.57933 , ...,  79.265945,
          83.73689 ,  81.55528 ],
        [ 94.32059 ,  87.616135,  87.72722 , ...,  78.72259 ,
          83.523506,  81.402435],
        ...,
        [ 94.75214 ,  87.993065,  85.203674, ..., 101.

In [93]:
# Evaluate ensemble model(s) predictions
ensemble_results = evaluate_preds(y_true=tf.squeeze(test_labels),
                                  y_pred=np.median(ensemble_preds, axis=0)) # take the median across all ensemble predictions
ensemble_results

{'mae': <tf.Tensor: shape=(), dtype=float32, numpy=6.5977607>,
 'mse': <tf.Tensor: shape=(), dtype=float32, numpy=97.955444>,
 'rmse': <tf.Tensor: shape=(), dtype=float32, numpy=7.7654915>,
 'mape': <tf.Tensor: shape=(), dtype=float32, numpy=8.418934>,
 'mase': <tf.Tensor: shape=(), dtype=float32, numpy=7.2521114>}

In [94]:
model_1_results

{'mae': <tf.Tensor: shape=(), dtype=float32, numpy=6.0471997>,
 'mse': <tf.Tensor: shape=(), dtype=float32, numpy=73.70388>,
 'rmse': <tf.Tensor: shape=(), dtype=float32, numpy=7.2709985>,
 'mape': <tf.Tensor: shape=(), dtype=float32, numpy=7.671045>,
 'mase': <tf.Tensor: shape=(), dtype=float32, numpy=6.6469483>}

In [95]:
model_0_results

{'mae': <tf.Tensor: shape=(), dtype=float32, numpy=11.429537>,
 'mse': <tf.Tensor: shape=(), dtype=float32, numpy=159.11499>,
 'rmse': <tf.Tensor: shape=(), dtype=float32, numpy=12.430287>,
 'mape': <tf.Tensor: shape=(), dtype=float32, numpy=14.108284>,
 'mase': <tf.Tensor: shape=(), dtype=float32, numpy=12.563094>}

In [98]:
model_1.save_weights('model_experiments/model_1_NBEATS.weights.h5')

In [99]:
# 1. Setup N-BEATS Block layer
nbeats_block_layer = NBeatsBlock(input_size=60,
                                 theta_size=THETA_SIZE,
                                 horizon=HORIZON,
                                 n_neurons=N_NEURONS,
                                 n_layers=N_LAYERS,
                                 name="InitialBlock")

# 2. Create input to stacks
stack_input = layers.Input(shape=[60], name="stack_input")

# 3. Create initial backcast and forecast input (backwards predictions are referred to as residuals in the paper)
backcast, forecast = nbeats_block_layer(stack_input)
residuals = layers.subtract([stack_input, backcast], name=f"subtract_00") 

# 4. Create stacks of blocks
for i, _ in enumerate(range(N_STACKS-1)): # first stack is already creted in (3)

    # 5. Use the NBeatsBlock to calculate the backcast as well as block forecast
    backcast, block_forecast = NBeatsBlock(
        input_size=60,
        theta_size=THETA_SIZE,
        horizon=HORIZON,
        n_neurons=N_NEURONS,
        n_layers=N_LAYERS,
        name=f"NBeatsBlock_{i}"
    )(residuals) # pass it in residuals (the backcast)

    # 6. Create the double residual stacking
    residuals = layers.subtract([residuals, backcast], name=f"subtract_{i}") 
    forecast = layers.add([forecast, block_forecast], name=f"add_{i}")

# 7. Put the stack model together
loaded_model = keras.Model(inputs=stack_input, 
                         outputs=forecast, 
                         name="model_1_N-BEATS")

In [102]:
# Make predictions with N-BEATS model
loaded_model_preds = loaded_model.predict(test_dataset)
# Evaluate N-BEATS model predictions
loaded_model_results = evaluate_preds(y_true=tf.squeeze(test_labels),
                                 y_pred=loaded_model_preds)
loaded_model_results

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


{'mae': <tf.Tensor: shape=(), dtype=float32, numpy=76.3029>,
 'mse': <tf.Tensor: shape=(), dtype=float32, numpy=7032.9097>,
 'rmse': <tf.Tensor: shape=(), dtype=float32, numpy=83.81564>,
 'mape': <tf.Tensor: shape=(), dtype=float32, numpy=92.027596>,
 'mase': <tf.Tensor: shape=(), dtype=float32, numpy=83.87045>}

In [103]:
loaded_model.load_weights('model_experiments/model_1_NBEATS.weights.h5')

In [104]:
# Make predictions with N-BEATS model
loaded_model_preds = loaded_model.predict(test_dataset)
# Evaluate N-BEATS model predictions
loaded_model_results = evaluate_preds(y_true=tf.squeeze(test_labels),
                                 y_pred=loaded_model_preds)
loaded_model_results

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step


{'mae': <tf.Tensor: shape=(), dtype=float32, numpy=6.0471997>,
 'mse': <tf.Tensor: shape=(), dtype=float32, numpy=73.70388>,
 'rmse': <tf.Tensor: shape=(), dtype=float32, numpy=7.2709985>,
 'mape': <tf.Tensor: shape=(), dtype=float32, numpy=7.671045>,
 'mase': <tf.Tensor: shape=(), dtype=float32, numpy=6.6469483>}

In [106]:
loaded_model_results is model_1_results

True